# Querying chpaf MoPo

In [1]:
import rdflib
import pandas as pd

# Set pandas to display wide columns
pd.set_option('max_colwidth', 1000)

g = rdflib.Graph()
g.parse("../examples/mopo.ttl")

<Graph identifier=Nfd1d820f6a8f4e80accf53aa987cb8e0 (<class 'rdflib.graph.Graph'>)>

## Chapter 1

In [9]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?department ?office ?id ?title ?description ?answer WHERE {

    ?registration_activity paf:activityType chpaf:Registration.

    ?registration_entity prov:wasGeneratedBy ?registration_activity;
        schema:identifier ?id;
        schema:title ?title;
        schema:description ?description;
        chpaf:department ?department;
        chpaf:office ?office.

    ?proposal_creation_activity prov:wasInformedBy+ ?registration_activity.

    ?proposal_entity prov:wasGeneratedBy ?proposal_creation_activity;
        chpaf:proposal ?answer.

    ?proposal_activity prov:wasInformedBy ?proposal_creation_activity.

    OPTIONAL {
    
        ?decision_activity prov:wasInformedBy ?proposal_activity;
            prov:startedAtTime ?decision_time.
    
    }

    FILTER (!BOUND(?decision_time) || ?decision_time >= "2023-01-01"^^xsd:date)


    FILTER (lang(?title) = "de")
    FILTER (lang(?description) = "de")
    FILTER (lang(?answer) = "de")

} ORDER BY ?department ?office

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,department,office,id,title,description,answer
0,https://ld.admin.ch/FCh,nan,18.4276,Erleichterter Informationsaustausch durch die Einführung von elektronischen Schnittstellen in der Bundesverwaltung,"Der Bundesrat wird beauftragt, den direkten Informationsaustausch innerhalb der Bundesverwaltung sowie zwischen Bundesverwaltung und Unternehmen beziehungsweise Einwohnerinnen und Einwohnern mittels Schaffung von elektronischen Schnittstellen bis spätestens 2022 zu ermöglichen.","Dem Anliegen der Motionen wurde mit dem Bundesgesetz über den Einsatz elektronischer Mittel zur Erfüllung von Behördenaufgaben (EMBAG) entsprochen, welches per 1. Januar 2024 in Kraft treten wird. Gemäss Artikel 13 EMBAG müssen die Behörden sicherstellen, dass sie den Datenaustausch über Schnittstellen abwickeln können. Der Bund führt über die Plattform I14Y eine Übersicht über die verfügbaren Schnittstellen. 2024 ist das Thema API zudem ein Fokusthema der Strategie Digitale Schweiz.\r\nDer Bundesrat erachtet die Anliegen der Motionen als erfüllt und beantragt deren Abschreibung."
1,https://ld.admin.ch/FCh,nan,19.4390,Anlaufstelle(n) in Sachen Administrativ- und Disziplinaruntersuchungen,"Der Bundesrat wird beauftragt, eine oder mehrere Stellen zu bezeichnen, welche über die erforderlichen Verfahrenskenntnisse bezüglich Administrativ- und Disziplinaruntersuchungen verfügen, sich über den aktuellen Wissensstand und die Rechtsprechung in diesem Bereich auf dem Laufenden halten und dadurch bei Bedarf anderen Einheiten des Bundes Rechtsauskünfte erteilen und diese beraten können. Er soll überdies dafür sorgen, dass sich die durchführenden Stellen bei formellen und rechtlichen Fragen systematischer an diese Beratungsstelle(n) wenden.",Die am 1. Januar 2022 in Kraft getretenen Weisungen des Bundesrates vom 18. August 2021 über Administrativ- und Disziplinaruntersuchungen (BBl 2021 1903) bezeichnen Beratungsstellen in Sachen Administrativ- und Disziplinaruntersuchungen und legen die Konsultations- und Dokumentationspflichten der für die Anordnung dieser Untersuchungen zuständigen Stellen fest. Diese Weisungen sind für die Departemente und die ihnen unterstellten Verwaltungseinheiten sowie die Bundeskanzlei verbindlich und werden in der Praxis umgesetzt.\r\nDer Bundesrat erachtet das Anliegen der Motion als erfüllt und beantragt deren Abschreibung.
2,https://ld.admin.ch/FCh,nan,20.3240,Covid-19. Bildung eines Zentrums für Innovation in der Bundesverwaltung (Public Innovation Hub),"Der Bundesrat wird beauftragt, einen Bericht zur Schaffung eines ""Public Innovation Hubs"", in Verbindung mit einer Innovationsstrategie für die Verwaltung, vorzulegen. Ziel des Hubs ist die Verknüpfung von ""Innovationsschaffern"" mit der Verwaltung, damit innovative Ideen geprüft und allenfalls rasch umgesetzt werden können. Der Bericht soll auf folgende Punkte eingehen: \r\n1. Trägerschaft: Der Hub ist interdepartemental auszugestalten. Die Interessen der Kantone werden berücksichtigt.\r\n2. Vernetzung: Der Hub ist als Schnittstelle zwischen Verwaltung auf der einen und Forschung und Tech-Unternehmen auf der anderen Seite zu konzipieren.\r\n3. Funktionsweise: Dank des Hubs sollen innovative Ideen rasch und unkompliziert getestet, wo nötig mit ""Pilotnormen"" temporär ermöglicht und im Erfolgsfall verwaltungsweit (und gegebenenfalls über alle föderalen Ebenen) verbreitet und implementiert werden können.",Postulatsbericht vom 23. September 2022 «Covid 19. Bildung eines Zentrums für Innovation in der Bundesverwaltung (Public Innovation Hub)». \r\nDer Bundesrat erachtet das Anliegen des Postulats als erfüllt und beantragt dessen Abschreibung
3,https://ld.admin.ch/FCh,nan,20.3280,Wissenschaftliches Potenzial für Krisenzeiten nutzen,"Der Bundesrat wird beauftragt, in einem Bericht zu prüfen, wie ein interdisziplinäres wissenschaftliches Netzwerk oder Kompetenzzentrum für Krisenlagen geschaffen werden kann. Der Bericht soll die entsprechenden Vor- und Nachteile, mögliche Umsetzungsoptione

## MoPo with Changed Registration

In [3]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?new ?old WHERE {

    ?new prov:wasInformedBy+ ?old;
        paf:activityType chpaf:Registration .
    
    ?old paf:activityType chpaf:Registration .

}

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,new,old
0,https://politics.ld.admin.ch/curia/22.3506/2023/registration-activity,https://politics.ld.admin.ch/curia/22.3506/2022/registration-activity
1,https://politics.ld.admin.ch/curia/22.3507/2023/registration-activity,https://politics.ld.admin.ch/curia/22.3507/2022/registration-activity
2,https://politics.ld.admin.ch/curia/22.3508/2023/registration-activity,https://politics.ld.admin.ch/curia/22.3508/2022/registration-activity
3,https://politics.ld.admin.ch/curia/22.3509/2023/registration-activity,https://politics.ld.admin.ch/curia/22.3509/2022/registration-activity


or faster if `paf:wasReplacedBy` predicate is set:

In [4]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?new ?old WHERE {

    ?old prov:wasGeneratedBy/paf:activityType chpaf:Registration.

    ?old paf:wasReplacedBy ?new.

}

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,new,old
0,https://politics.ld.admin.ch/curia/22.3506/2023/registration-entity,https://politics.ld.admin.ch/curia/22.3506/2022/registration-entity
1,https://politics.ld.admin.ch/curia/22.3507/2023/registration-entity,https://politics.ld.admin.ch/curia/22.3507/2022/registration-entity
2,https://politics.ld.admin.ch/curia/22.3508/2023/registration-entity,https://politics.ld.admin.ch/curia/22.3508/2022/registration-entity
3,https://politics.ld.admin.ch/curia/22.3509/2023/registration-entity,https://politics.ld.admin.ch/curia/22.3509/2022/registration-entity


## All Activities for a Certain Curia Number

In [5]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?id ?activity WHERE {

    BIND ("02.3532" as ?id)

    ?registration_entity schema:identifier ?id;
        prov:wasGeneratedBy ?registration_activity.

    ?activity prov:wasInformedBy* ?registration_activity;
        

}

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,id,activity
0,02.3532,https://politics.ld.admin.ch/curia/02.3532/2021/registration-activity
1,02.3532,https://politics.ld.admin.ch/curia/02.3532/2021/information-creation-activity
2,02.3532,https://politics.ld.admin.ch/curia/02.3532/2021/information-activity
3,02.3532,https://politics.ld.admin.ch/curia/02.3532/2022/information-creation-activity
4,02.3532,https://politics.ld.admin.ch/curia/02.3532/2022/information-activity
5,02.3532,https://politics.ld.admin.ch/curia/02.3532/2023/information-creation-activity
6,02.3532,https://politics.ld.admin.ch/curia/02.3532/2023/information-activity


## Proposal after Information

In [6]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT * WHERE {

    ?proposal_activity paf:activityType chpaf:Proposal;
        prov:wasInformedBy/prov:wasInformedBy ?information_activity.

    ?information_activity paf:activityType chpaf:Information.
        

}

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,information_activity,proposal_activity
0,https://politics.ld.admin.ch/curia/00.3536/2021/information-activity,https://politics.ld.admin.ch/curia/00.3536/2022/proposal-activity
1,https://politics.ld.admin.ch/curia/06.3009/2022/information-activity,https://politics.ld.admin.ch/curia/06.3009/2023/proposal-activity
2,https://politics.ld.admin.ch/curia/08.3493/2021/information-activity,https://politics.ld.admin.ch/curia/08.3493/2022/proposal-activity
3,https://politics.ld.admin.ch/curia/09.3089/2022/information-activity,https://politics.ld.admin.ch/curia/09.3089/2023/proposal-activity
4,https://politics.ld.admin.ch/curia/09.3535/2022/information-activity,https://politics.ld.admin.ch/curia/09.3535/2023/proposal-activity
...,...,...
112,https://politics.ld.admin.ch/curia/20.3886/2022/information-activity,https://politics.ld.admin.ch/curia/20.3886/2023/proposal-activity
113,https://politics.ld.admin.ch/curia/20.3921/2022/information-activity,https://politics.ld.admin.ch/curia/20.3921/2023/proposal-activity
114,https://politics.ld.admin.ch/curia/20.3933/2022/information-activity,https://politics.ld.admin.ch/curia/20.3933/2023/proposal-activity
115,https://politics.ld.admin.ch/curia/20.4211/2022/information-activity,https://politics.ld.admin.ch/curia/20.4211/2023/proposal-activity


## Registration Info in Correct Order

With the `values` keyword it can also be selected, which predicates to show.

In [7]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?p ?o WHERE {

    BIND ("02.3532" as ?id)

    ?registration_entity schema:identifier ?id;
        ?p ?o.
        
    values (?p ?order) { (schema:identifier 1) (schema:title 2) (schema:author 3) (schema:description 4) }
    
} ORDER BY ?order

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,p,o
0,http://schema.org/identifier,02.3532
1,http://schema.org/title,Bestimmungen über Architektur- und Bauleistungen im OR. Schutz der Auftraggebenden
2,http://schema.org/title,Dispositions du CO relatives au bâtiment. Protection du maître de l'ouvrage
3,http://schema.org/title,Disposizioni del CO in materia edilizia. Protezione del committente
4,http://schema.org/author,Hildegard Fässler-Osterwalder
5,http://schema.org/description,"Der Bundesrat wird beauftragt, die Bestimmungen über Architektur- und Baudienstleistungen im Obligationenrecht (OR) unter einem neuen Titel zusammenzufassen, zeitgemäss zu formulieren und wie folgt zu ergänzen:\r\n1. Rüge-, Garantie- und Haftungsfristen sind im Gesetz präzis zu regeln. Das heutige komplizierte Verfahren zur Unterbrechung der Verjährungsfrist bei Nichterledigung rechtzeitig gerügter Mängel ist zu vereinfachen.\r\n2. Zur Sicherstellung von Garantie- und Haftungsansprüchen der Auftraggeberschaft haben Auftragnehmende eine Bank- und Versicherungsgarantie vorzuweisen.\r\n3. Die Anforderungen an einen Generalunternehmervertrag und allgemein an Verträge zu Fest- oder Pauschalpreisen, die unterschiedliche Dienstleistungen beinhalten, sind klar zu umschreiben. Insbesondere ist festzuhalten, dass in solchen Fällen der Anbieter bzw. die Anbieterin auch nach Fertigstellung für die Behebung von Mängeln verantwortlich ist.\r\n4. Architekturleistungen sind analog zu werkvertragli..."
6,http://schema.org/description,"Le Conseil fédéral est chargé de regrouper sous un seul et même titre du Code des obligations (CO) toutes les dispositions concernant les prestations fournies dans les domaines de la construction et de l'architecture, de reformuler ces dispositions pour les rendre plus modernes et de les compléter comme suit:\r\n1. Les délais de réclamation et de garantie ainsi que la durée pendant laquelle la responsabilité s'applique devront être réglés de manière précise dans la loi; il y aura lieu notamment de simplifier la procédure actuelle - fort compliquée - qui permet d'interrompre le délai de prescription lorsqu'un vice signalé à temps n'a pas été réparé.\r\n2. Dans le but de protéger le droit de garantie du mandant ainsi que son droit d'invoquer la responsabilité du mandataire, celui-ci devra désormais produire une garantie bancaire ou une garantie de son assurance.\r\n3. Les exigences à remplir dans le cadre de contrats d'entreprise générale ou de contrats prévoyant un prix fixe ou un p..."
7,http://schema.org/description,"Le regole applicabili ai contratti di costruzione e di architetto sono contenute nel Codice delle obbligazioni, nella giurisprudenza, nella dottrina nonché nel Regolamento SIA 102 e nella Norma SIA 118, se integrati al contratto. Ne risulta una regolamentazione complessa che può anche rivelarsi inadeguata, poiché la qualifica di mandato o di contratto d'appalto delle prestazioni di costruzione e di architettura non offre sempre soluzioni soddisfacenti. Ciò si verifica in particolare quando le diverse fasi del progetto di costruzione sono definite a un prezzo fisso nel quadro di un contratto di appalto generale. La situazione attuale ha anche un vantaggio da non sottovalutare: garantisce flessibilità in un settore complesso e tecnico con numerosi attori (committente, architetti, ingegneri) che forniscono diversi tipi di prestazioni combinabili tra loro (pianificazione, direzione parziale o globale dei lavori ed esecuzione degli stessi). Per tali motivi la questione relativa a una re..."
